对poi类型分类

df['pointx_class']里都是编号，例如3170240.,  3170245.,  6340457.,  6340454.,  3190257。我需要对df增加一列'poi_class',里面填每一行的分类属性。 下面是所有分类属性，格式为 分类属性：从左开始匹配的编号。意思是，只要这个数字匹配到编号从左开始的几位数，那么这一行的分类属性就是它；如果没有匹配的分类属性，那么就填为other. 

Accommodation: 101

Eating and drinking：102,947：

service：2

culture and tourism：3

entertainment：4

education：527,531,532

health：526,528,529,

Manufacturing and production: 7

shopping：946，948，949

finance：209 （这个在gpt给的匹配规则中会把service再改为finance，很聪明）

office： 633，635

Infrastructure and facilities ：634

Airports：1053

Public transport ：1057,1059

Road and rail ：1054,1055

Water transport: 1056

In [1]:
import os
import pandas as pd


df = pd.read_csv(r"D:\POI data\poi_cleaned_data\poi_deleted_columns.csv")
df

,name,pointx_class,latitude,longitude
0,Roman Road,3170240.0,51.435388,-0.505500
1,Moat,3170245.0,51.325649,0.174568
2,Moat,3170245.0,51.515484,0.246527
3,Moat,3170245.0,51.497405,0.258403
4,Moat,3170245.0,51.386622,0.161935
...,...,...,...,...
375479,Abbey Community Centre,6340456.0,51.540131,0.071803
375480,Scouts,6350452.0,51.563827,-0.442227
375481,St Albans Church Hall,6340456.0,51.582993,-0.367225
375482,Brickfield Community Centre,6340456.0,51.482459,-0.314747


In [44]:
# df['pointx_class'].unique()

In [2]:
def match_poi_class(number):
    if str(number).startswith(('102', '947')):
        return 'Eating and drinking'
    elif str(number).startswith('101'):
        return 'Accommodation'
    elif str(number).startswith('209'):
        return 'Finance'
    elif str(number).startswith('2'):
        return 'Service'
    elif str(number).startswith('3'):
        return 'Culture and tourism'
    elif str(number).startswith('4'):
        return 'Entertainment'
    elif str(number).startswith(('527', '531', '532')):
        return 'Education'
    elif str(number).startswith(('526', '528', '529')):
        return 'Health'
    elif str(number).startswith(('946', '948', '949')):
        return 'Shopping'
    
    elif str(number).startswith(('633', '635')):
        return 'Office'
    elif str(number).startswith('634'):
        return 'Infrastructure and facilities'
    elif str(number).startswith('7'):
        return 'Manufacturing and production'
    elif str(number).startswith('1053'):
        return 'Airports'
    elif str(number).startswith(('1057', '1059')):
        return 'Public transport'
    elif str(number).startswith(('1054', '1055')):
        return 'Road and rail'
    elif str(number).startswith('1056'):
        return 'Water transport'
    else:
        return 'Other'

df['poi_class'] = df['pointx_class'].apply(match_poi_class)


In [3]:
count_by_poi_class = df['poi_class'].value_counts()
count_by_poi_class


poi_class
Service                          123520
Eating and drinking               56356
Infrastructure and facilities     49504
Shopping                          26741
Public transport                  25219
Health                            17907
Manufacturing and production      15606
Entertainment                     12776
Education                         12335
Road and rail                     12310
Culture and tourism               11101
Office                             7704
Water transport                    2441
Accommodation                      1926
Airports                             38
Name: count, dtype: int64

In [4]:
other_class = df[df['poi_class']=='Other']
other_class['pointx_class'].unique()

array([], dtype=float64)

In [48]:
other_class

,name,pointx_class,latitude,longitude,poi_class


In [5]:
poi_class_unique = df['poi_class'].unique()
poi_class_unique.size

15

为了将'poi_class'列中的属性拆分成新的列，并且如果该行中'poi_class'列的属性与新列名一致，则将相应的列中的值设为1，否则设为0

In [6]:
dummies = pd.get_dummies(df['poi_class'])
df = pd.concat([df, dummies], axis=1)

for column in dummies.columns:
    df[column] = df.apply(lambda row: 1 if row['poi_class'] == column else 0, axis=1)



df

KeyboardInterrupt: 

In [55]:
df_col = df.columns
df_col

Index(['name', 'pointx_class', 'latitude', 'longitude', 'poi_class',
       'Accommodation', 'Airports', 'Culture and tourism',
       'Eating and drinking', 'Education', 'Entertainment', 'Health',
       'Infrastructure and facilities', 'Manufacturing and production',
       'Office', 'Public transport', 'Road and rail', 'Service', 'Shopping',
       'Water transport'],
      dtype='object')

In [ ]:
df = df.rename(columns={'latitude': 'lat'})
df = df.rename(columns={'longitude': 'lon'})

In [56]:
df.to_csv(r"D:\POI data\poi_cleaned_data\poi_has_classes.csv", index=False)